In [ ]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
import random
%matplotlib inline

env_closer = closer.Closer()

In [ ]:
class CorningEnv(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    
    def __init__(self, camera,
                     Width = 417,
                     Height = 404,
                     threshold = 85,
                     file_path = r'C:\Users\CIG\Documents\corning_basler_RL\single_agent'):
        
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        self.camera = camera
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.Width = Width
        self.Height = Height
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        
        # environment parameters 
        # coarse V
        # fine V
        # 
        self.action_space = spaces.MultiDiscrete([69, 99, number_of_actions])
        
        # image
        self.observation_space = spaces.Box(low=0, 
                                            high=255, 
                                            shape=(Height, Width, 3),
                                            dtype = np.uint8)
        
        
        self.threshold = threshold
        self.file_path = file_path
        self.i = 0
        time.sleep(1)
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
    
    def basler(self, action):
        
        ExposureTimeRaw = int(self.range_ex_time[action])
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(5)
        
    def lens_movement(self, action):
        
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(2)
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                break
        self.camera.StopGrabbing()
        return image
    
    def focus_value(self, img):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(img),cv2.CV_64F, 0, 1, ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
    
    
    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        # https://docs.baslerweb.com/features
        self.camera.ExposureTimeRaw.Value = 20000
        
        self.lens_movement(24.0)
        
        self.i = 0
        time.sleep(5)
        
        observation = self.grab_image().astype(np.uint8)
        
        return observation
    
    def check_hist(self, img):
        
        hist , bin_edges = np.histogram(np.array(img).ravel())
        #reward = np.sum(bin_edges) - 250
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin > 50 and max_value_bin < 170:
            pic_ok = True
        else:
            pic_ok = False
        return pic_ok
        
        
    def step(self, action):
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        print(action)
        
        action_lens_coarse = 0
        action_lens_fine = action[1]
        if action[0] < 24:
            action_lens_coarse = 24
        else:
            action_lens_coarse = action[0]
        action_lens = float("%s.%s"% (str(action_lens_coarse), str(action_lens_fine)))
        
        action_camera = action[2]
        self.basler(action_camera)
        self.lens_movement(action_lens)
        
        observation = self.grab_image()
        
        is_it_ok = self.check_hist(observation)
        
        prediction_yolo = self.model_yolo(observation)
        final_result_yolo = self.find_class(prediction_yolo)
        
        if final_result_yolo.size == 0:
            print('not detected')
            foc_value = -1
            betta = 0.5
            reward = -100-betta*self.i
            
        else:
            image = Image.fromarray(observation)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            foc_value = self.focus_value(image)
            
        if foc_value > self.threshold:
            done = True
            reward = 1
            print('------',True, action, self.i, '------')
        else:
            done = False
            reward = 1 - 0.001*(foc_value - self.threshold)**2
            print(False, action)
            
        im = Image.fromarray(observation)
        filename = self.file_path + "\img_%s_%s_%s.png" % (str(action_lens), 
                                                           str(self.range_ex_time[action_camera]), 
                                                           str(done))
        im.save(filename)
        self.i+=1
        
        if is_it_ok == True:
            reward +=1
        else:
            reward -= 10
                
        return observation.astype(np.uint8), reward, done, {}
    
    

In [ ]:
# Create an instant camera object with the camera device found first.
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [ ]:
env = CorningEnv(camera)

In [ ]:
# from stable_baselines.common.env_checker import check_env
# # It will check your custom environment and output additional warnings if needed
# check_env(env)
#env.action_space.low

In [ ]:
from stable_baselines.common.vec_env import DummyVecEnv
# model = PPO2(MlpPolicy, env, 
#              #learning_rate = sched_LR.value,
#              verbose=1, 
#              tensorboard_log = '')

# Load the trained agent
model = PPO2.load("model_corning_basler_single_agent_03", 
                  env=DummyVecEnv([lambda: env]), 
                  tensorboard_log = '')

In [ ]:
# Enjoy trained agent
# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)

In [ ]:
model.learn(total_timesteps=5000, tb_log_name="corning_basler_single_agent_05")#, reset_num_timesteps=False)

In [ ]:
#Сохранить модель: 

#model.save("model_corning_basler_single_agent_03")

In [ ]:
#tensorboard --logdir=C:\\Users\\CIG\\Documents\\MATLAB